# Soccer and Statistics: Modeling the Group Stage

In [1]:
import numpy as np
import pandas as pd
from collections import defaultdict
import string
from soccer_functions import standings, standings_n_teams, sequences

In the first part of a soccer tournament (like the World Cup) teams are often divided into groups, and every team in that group will play every other team in that group. The results of this "round-robin" play will determine the teams to advance to the single-elimination tournament (or "knockout round"). A team will earn three points for a win and one point for a tie.

Very often a group will comprise four teams. For a four-team group, how many different game outcomes are possible? How many different sets of final standings are possible? **How many different sets of final standings are possible if we abstract away from the team identities?**

This last question is what I want to pursue here. As we shall see, the complexity of this problem grows quickly as we increase the number of teams in our group.

## Two Teams

If we had only two teams in our group, then there would of course be only one game to play ($2\choose 2$$ = 1$). The game could result either in a win for one team and a loss for the other, or it could result in a tie. In the first case we'd have a set of final standings like:

Team | Points
-|-
Team A | 3
Team B | 0

In the second case we'd have a set of final standings like:

Team | Points
-|-
Team A | 1
Team B | 1

Of course, we could also have:

Team | Points
-|-
Team B | 3
Team A | 0

But if we abstract away from the team identities, then this result is no different from the first. So for two teams there are only two possible sets of final standings: 3-0 and 1-1.

## Three Teams

The problem is not particularly interesting with only two teams, but observe how much more complex the situation is with three. If we had three teams in our group, then there would be three games to play ($3\choose 2$$ = 3$). Now there are three games (Team A vs. Team B, Team A vs. Team C, and Team B vs. Team C), each with three potential outcomes (win, lose, or draw). So that makes for 27 different possible scenarios.

Let's take a look at the possibilities:

I'll use a three-character string to represent the results of our three games. The first character will represent the result of the game played between Teams A and B, the second character will represent the result of the game played between Teams A and C, and the last character will represent the result of the game played between Teams B and C. I'll use a lowercase 'a' to represent a win by Team A (and *mutatis mutandis* for the other two teams). A tie will be represented with a 't'.

Thus our 27 possibilities are:

1. 'aab'
2. 'aac'
3. 'aat'
4. 'acb'
5. 'acc'
6. 'act'
7. 'atb'
8. 'atc'
9. 'att'
10. 'bab'
11. 'bac'
12. 'bat'
13. 'bcb'
14. 'bcc'
15. 'bct'
16. 'btb'
17. 'btc'
18. 'btt'
19. 'tab'
20. 'tac'
21. 'tat'
22. 'tcb'
23. 'tcc'
24. 'tct'
25. 'ttb'
26. 'ttc'
27. 'ttt'

Each of these sets of game outcomes corresponds to a final set of standings. Let's take the first: The string 'aab' represents the possible world where:

- Team A beats Team B,
- Team A beats Team C, and
- Team B beats Team C.

In that case, our final standings are:

Team | Points
-|-
A | 6
B | 3
C | 0

Or, abstracting away from the team identities, we have 6-3-0.

### Unique Sets of Final Standings for Three Teams

How many sets of final standings are possible for three teams? If we look at each set of final standings represented by the outcome strings above, we'll find that there are seven unique possibilities:

- 'aab' $\rightarrow$ 6-3-0
- 'aac' $\rightarrow$ 6-3-0
- 'aat' $\rightarrow$ 6-1-1
- 'acb' $\rightarrow$ 3-3-3
- 'acc' $\rightarrow$ 6-3-0
- 'act' $\rightarrow$ 4-3-1
- 'atb' $\rightarrow$ 4-3-1
- 'atc' $\rightarrow$ 4-4-0
- 'att' $\rightarrow$ 4-2-1
- 'bab' $\rightarrow$ 6-3-0
- 'bac' $\rightarrow$ 3-3-3
- 'bat' $\rightarrow$ 4-3-1
- 'bcb' $\rightarrow$ 6-3-0
- 'bcc' $\rightarrow$ 6-3-0
- 'bct' $\rightarrow$ 4-4-0
- 'btb' $\rightarrow$ 6-1-1
- 'btc' $\rightarrow$ 4-3-1
- 'btt' $\rightarrow$ 4-2-1
- 'tab' $\rightarrow$ 4-4-0
- 'tac' $\rightarrow$ 4-3-1
- 'tat' $\rightarrow$ 4-2-1
- 'tcb' $\rightarrow$ 4-3-1
- 'tcc' $\rightarrow$ 6-1-1
- 'tct' $\rightarrow$ 4-2-1
- 'ttb' $\rightarrow$ 4-2-1
- 'ttc' $\rightarrow$ 4-2-1
- 'ttt' $\rightarrow$ 2-2-2

The seven unique possibilities are:

- 6-3-0
- 6-1-1
- 4-4-0
- 4-3-1
- 4-2-1
- 3-3-3
- 2-2-2

## Four Teams

The case of three teams is considerably more complex than the case of two. For four teams we had better try to automate some of our work so that we don't need to write all the possibilities out by hand. In fact, with four teams there are now six games to be played, and so that makes for $3^6 = 729$ possible outcomes!

Let's bring in our helper function that will output the standings for a given outcome string:

In [2]:
# Let's test out our function for the following possible world:
# - Team A beats Team B,
# - Team A beats Team C,
# - Team A beats Team D,
# - Team B and Team C tie,
# - Team B and Team D tie, and
# - Team C and Team D tie.
# We expect the function to show us a set of standings where
# Team A has 9 points and the other teams have 2 each.

standings('aaattt')

,points
a,9
b,2
c,2
d,2


In [3]:
# Now let's simulate all 729 possible worlds by creating all
# of the relevant outcome strings:

coll = []

for first in ('abt'):
    for second in ('act'):
        for third in ('adt'):
            for fourth in ('bct'):
                for fifth in ('bdt'):
                    for sixth in ('cdt'):
                        coll.append(standings(first+second+third+fourth+fifth+sixth))

len(coll)

729

In [4]:
# We now have a list of 729 DataFrames. Let's look at the first:
coll[0]

,points
a,9
b,6
c,3
d,0


In [5]:
# These DataFrames may prove unwieldy for our purposes. Let's
# make a large NumPy array instead:

big_arr = np.concatenate([df.values for df in coll], axis=1)

big_arr

array([[9, 9, 9, ..., 5, 5, 3],
       [6, 6, 6, ..., 3, 3, 3],
       [3, 3, 1, ..., 3, 3, 3],
       [0, 0, 1, ..., 2, 2, 3]])

In [6]:
# The first two outcome strings ('aaabbc' and 'aaabbd') result in the
# same standings: 9-6-3-0:

big_arr[:, 0] ==  big_arr[:, 1]

array([ True,  True,  True,  True])

In [7]:
# Let's now use defaultdict to count how many of each set of final
# standings we have:

counter = defaultdict(int)

# Technical note: We can't use the NumPy arrays themselves as keys,
# since they're not hashable. So we'll turn them into strings first!
for stand in big_arr.T:
    counter[str(stand)] += 1

counter

defaultdict(int,
            {'[9 6 3 0]': 24,
             '[9 6 1 1]': 12,
             '[9 3 3 3]': 8,
             '[9 4 3 1]': 24,
             '[9 4 4 0]': 12,
             '[9 4 2 1]': 24,
             '[9 2 2 2]': 4,
             '[6 6 3 3]': 24,
             '[6 6 6 0]': 8,
             '[6 6 4 1]': 24,
             '[7 6 3 1]': 24,
             '[6 4 4 3]': 36,
             '[7 6 4 0]': 24,
             '[6 5 4 1]': 24,
             '[6 4 4 2]': 24,
             '[7 6 2 1]': 24,
             '[6 5 2 2]': 12,
             '[7 4 3 3]': 24,
             '[7 7 3 0]': 12,
             '[7 5 3 1]': 24,
             '[7 4 3 2]': 24,
             '[7 5 4 0]': 24,
             '[7 4 3 1]': 24,
             '[7 4 4 1]': 36,
             '[7 4 2 2]': 24,
             '[7 7 1 1]': 6,
             '[7 5 2 1]': 24,
             '[7 3 2 2]': 12,
             '[5 4 4 3]': 24,
             '[5 5 3 2]': 12,
             '[4 4 4 3]': 8,
             '[4 4 4 4]': 6,
             '[5 4 4 2]': 24,

### Unique Sets of Final Standings for Four Teams

The number of entries in our dictionary is the answer to our question about how many unique sets of final standings there are:

In [8]:
len(counter.keys())

40

In [9]:
# This should sum to 729:
sum(counter.values())

729

### Exploration of Possible Final Standings

Notice that, when a game results in a win for a team (and a loss for that team's opponent), that's a total of three points that will be recorded and added to the standings. But when a game results in a tie, that's a total of only two points that will be added.

It's therefore easy to group these possible final standings according to how many ties happened.

First let's create a NumPy array of the unique sets of final standings:

In [10]:
poss = np.unique(big_arr.T, axis=0)
poss

array([[3, 3, 3, 3],
       [4, 4, 4, 3],
       [4, 4, 4, 4],
       [5, 3, 3, 2],
       [5, 4, 3, 2],
       [5, 4, 4, 2],
       [5, 4, 4, 3],
       [5, 5, 2, 2],
       [5, 5, 3, 1],
       [5, 5, 3, 2],
       [5, 5, 4, 1],
       [5, 5, 5, 0],
       [6, 4, 4, 2],
       [6, 4, 4, 3],
       [6, 5, 2, 2],
       [6, 5, 4, 1],
       [6, 6, 3, 3],
       [6, 6, 4, 1],
       [6, 6, 6, 0],
       [7, 3, 2, 2],
       [7, 4, 2, 2],
       [7, 4, 3, 1],
       [7, 4, 3, 2],
       [7, 4, 3, 3],
       [7, 4, 4, 1],
       [7, 5, 2, 1],
       [7, 5, 3, 1],
       [7, 5, 4, 0],
       [7, 6, 2, 1],
       [7, 6, 3, 1],
       [7, 6, 4, 0],
       [7, 7, 1, 1],
       [7, 7, 3, 0],
       [9, 2, 2, 2],
       [9, 3, 3, 3],
       [9, 4, 2, 1],
       [9, 4, 3, 1],
       [9, 4, 4, 0],
       [9, 6, 1, 1],
       [9, 6, 3, 0]])

Now we'll check the possibilities for each number of ties between 0 and 6. If there are no ties, the total points will be 18. If there is exactly one tie, the total points will be 17. Etc.

In [11]:
# 6 ties
[rec for rec in poss if rec.sum() == 12]

[array([3, 3, 3, 3])]

In [12]:
# 5 ties
[rec for rec in poss if rec.sum() == 13]

[array([5, 3, 3, 2])]

In [13]:
# 4 ties
[rec for rec in poss if rec.sum() == 14]

[array([5, 4, 3, 2]),
 array([5, 5, 2, 2]),
 array([5, 5, 3, 1]),
 array([7, 3, 2, 2])]

In [14]:
# 3 ties
[rec for rec in poss if rec.sum() == 15]

[array([4, 4, 4, 3]),
 array([5, 4, 4, 2]),
 array([5, 5, 3, 2]),
 array([5, 5, 4, 1]),
 array([5, 5, 5, 0]),
 array([6, 5, 2, 2]),
 array([7, 4, 2, 2]),
 array([7, 4, 3, 1]),
 array([7, 5, 2, 1]),
 array([9, 2, 2, 2])]

In [15]:
# 2 ties
[rec for rec in poss if rec.sum() == 16]

[array([4, 4, 4, 4]),
 array([5, 4, 4, 3]),
 array([6, 4, 4, 2]),
 array([6, 5, 4, 1]),
 array([7, 4, 3, 2]),
 array([7, 4, 4, 1]),
 array([7, 5, 3, 1]),
 array([7, 5, 4, 0]),
 array([7, 6, 2, 1]),
 array([7, 7, 1, 1]),
 array([9, 4, 2, 1])]

In [16]:
# 1 tie
[rec for rec in poss if rec.sum() == 17]

[array([6, 4, 4, 3]),
 array([6, 6, 4, 1]),
 array([7, 4, 3, 3]),
 array([7, 6, 3, 1]),
 array([7, 6, 4, 0]),
 array([7, 7, 3, 0]),
 array([9, 4, 3, 1]),
 array([9, 4, 4, 0]),
 array([9, 6, 1, 1])]

In [17]:
# No ties
[rec for rec in poss if rec.sum() == 18]

[array([6, 6, 3, 3]),
 array([6, 6, 6, 0]),
 array([9, 3, 3, 3]),
 array([9, 6, 3, 0])]

In [18]:
# The 40 possible standings in a DataFrame:
df = pd.DataFrame(np.unique(big_arr.T, axis=0).T, columns=range(40, 0, -1))
df[df.columns[::-1]]

,1,2,3,4,5,6,7,8,9,10,...,31,32,33,34,35,36,37,38,39,40
0,9,9,9,9,9,9,9,7,7,7,...,5,5,5,5,5,5,5,4,4,3
1,6,6,4,4,4,3,2,7,7,6,...,5,5,5,4,4,4,3,4,4,3
2,3,1,4,3,2,3,2,3,1,4,...,3,3,2,4,4,3,3,4,4,3
3,0,1,0,1,1,3,2,0,1,0,...,2,1,2,3,2,2,2,4,3,3


If we count the point totals for each set of standings, we can count how many of each we have for a given number of ties:

In [19]:
df.sum().value_counts().reset_index().sort_values('index', ascending=False)

,index,0
3,18,4
2,17,9
0,16,11
1,15,10
4,14,4
5,13,1
6,12,1


## Larger Groups

For four teams there are 40 different sets of final standings. Dare we calculate the number for still larger groups?

We'll need our more general helper function for the case of a five-team group.

In [20]:
standings_n_teams('aaaabbbctt', n=5)

For 5 teams there will be 10 games and so 59049 possibilities.


array([12,  9,  4,  2,  1])

For fun, let's generate all the possibilities for a five-team group:

We're going to make use of a helper function here that will generate all possible outcome strings for an input array of strings that represent each game.

In [21]:
standings = np.zeros((5, 59049))

five_team_group = ['abt', 'act', 'adt', 'aet', 'bct',
                   'bdt', 'bet', 'cdt', 'cet', 'det']

results = sequences(five_team_group)

for j in range(59049):
    standings[:, j] = standings_n_teams(results[j],
                                        n=5,
                                       verbose=0)

In [22]:
standings[:, 0]

array([12.,  9.,  6.,  3.,  0.])

If we find the unique ones and then count the columns, that should be the answer to the question of how many unique sets of final standings there are for a five-team group:

In [23]:
np.unique(standings, axis=1).shape[1]

355

## Continuing the Sequence

If we build the sequence $a_n$ of sets of different possible standings, where $n$ represents the number of teams in our round-robin group, we have so far found:

Teams | Possible Final Standings
-|-
2 | 2
3 | 7
4 | 40
5 | 355

And it turns out that we have [predecessors](https://oeis.org/A064626) in the study of this sequence.